In [ ]:
!pip install tweepy

In [35]:
!pip install nltk

tensorflow 2.1.0 requires absl-py>=0.7.0, which is not installed.
tensorflow 2.1.0 requires astor>=0.6.0, which is not installed.
tensorflow 2.1.0 requires gast==0.2.2, which is not installed.
tensorflow 2.1.0 requires google-pasta>=0.1.6, which is not installed.
tensorflow 2.1.0 requires grpcio>=1.8.6, which is not installed.
tensorflow 2.1.0 requires keras-applications>=1.0.8, which is not installed.
tensorflow 2.1.0 requires keras-preprocessing>=1.1.0, which is not installed.
tensorflow 2.1.0 requires opt-einsum>=2.3.2, which is not installed.
tensorflow 2.1.0 requires protobuf>=3.8.0, which is not installed.
tensorflow 2.1.0 requires tensorflow-estimator<2.2.0,>=2.1.0rc0, which is not installed.
tensorflow 2.1.0 requires termcolor>=1.1.0, which is not installed.
tensorboard 2.1.0 requires absl-py>=0.4, which is not installed.
tensorboard 2.1.0 requires google-auth<2,>=1.6.3, which is not installed.
tensorboard 2.1.0 requires google-auth-oauthlib<0.5,>=0.4.1, which is not installed.

In [50]:
import os
import pandas as pd
import tweepy as tw
import seaborn as sns
import string
# import matplotlib.pyplot as plt

# import itertools
# import collections

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import re
import networkx

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")


In [64]:
from nltk.corpus import stopwords
import string

In [22]:
api_key="Hxh3XPIdMuvGi6CRhaJplZVQF"
api_secret= "ZHrw0QbqqBElOtdWbZK90s6QYTl6BUj24Aad0462W3KHTpZVAh"
access_token = "846412731926413317-GHRx2c3ANGqJkgBJIb9YJYhnQ4adBY3"
access_token_secret= "cke8pH4s2Eiexw1ln3N71dCbiAnhZO8mrDaddpVO0a94v"

## Autorização


In [23]:

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


## Funções de tratamento de dados

## Stopwords

In [68]:
punctuation = list(string.punctuation)
lista_stopwords  = set(stopwords.words('portuguese') + punctuation + ['rt', 'via'])



In [89]:
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).lower().split())

In [88]:
def set_tokenize(txt):
    return word_tokenize(txt)
    

In [90]:
def remove_stop_Words(txt,lista_stopwords):
    palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in lista_stopwords]

    return palavras_sem_stopwords

In [91]:
def pre_process(txt,lista_stopwords):
    remove = remove_url(txt)
    tokenize = set_tokenize(remove_url)
    filter_stopwords = remove_stop_Words(tokenize,lista_stopwords)

    return filter_stopwords


In [93]:
pre_process("NameError: name 'nltk' is not defined",lista_stopwords)

TypeError: expected string or bytes-like object

In [92]:
twttwes = [pre_process(tweet.text,lista_stopwords) for tweet in dados]
twttwes[:5]

TypeError: expected string or bytes-like object

## Variaveis

In [27]:
dados = []  # lista para guardar os twtters originais
data = '2019-11-01'
termos = ['STF']

In [30]:
search_term = "#climate+change -filter:retweets"

tweets = tweepy.Cursor(api.search,
                   q=search_term,
                   lang="en",
                   since='2018-11-01').items(1000)

for item in tweets:
    dados.append(item)                   


TweepError: Twitter error response: status code = 429

In [48]:
twttwes = [pre_process(tweet.text) for tweet in dados]
twttwes[:5]

['tourism establishments and management must be aware of climate threats says minister dogleydidier as seychelles',
 'cacrisalves rebuildmacro cambridgeinet kingscollege cambridgeuni cambridgezero esrc matthewagarwala',
 'joins us next week for an evening talk exploring whether existing codes of ethics remain appropriate in a climate',
 'theres a strange joy that comes with accepting the imminent catastrophe that is climate change perhaps when we',
 'hear the march 3 investor ideas climate change and cleantech podcast']